<a href="https://colab.research.google.com/github/AlexeyTri/PyTorchTutorials/blob/main/Tutorial8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers

---
transforms can be applied to PIL images, tensors, ndarrays, or custom data during creation of the DataSet


On Images

---
centerCrop, Grayscale, Pad, RandomAffine, RandomCrop, RandomHorizonalFlip, RandomRotation, Resize, Scale



On Tensor

---
LinearTransformation, Normalize, RandomErasing



Conversion

---
ToPILImage: from tensor or ndarray
ToTensor: from numpy.ndarray or PILImage



Generic

---
Use Lambda



Custom

---
write own class


Compose muptituple Transforms

---
composed = transforms.Compose([Rescale(256),
                                RandomCrop(224)]

In [85]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [64]:
# Clone the entire repo.
!git clone -l -s https://github.com/AlexeyTri/PyTorchTutorials.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.
/content/cloned-repo
README.md	 Tutorial2.ipynb  Tutorial4.ipynb  Tutorial6.ipynb
Tutorial1.ipynb  Tutorial3.ipynb  Tutorial5.ipynb  wine.csv


In [89]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('/content/cloned-repo/wine.csv', delimiter=',',dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # note that we do not convert to tensor here
        self.x_data = xy[:, 1:]
        self.y_data = xy[:, [0]]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.n_samples

In [87]:
class ToTensor:
    # convert ndarrays to Tensor
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [88]:
class MulTransform:
    # multiply inputs with a given factor
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

In [90]:
print('Without Transform')
dataset = WineDataset()
first_data = dataset[0]
features, labels = first_data
type(features), type(labels), features, labels

Without Transform


(numpy.ndarray,
 numpy.ndarray,
 array([1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
        3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
        1.065e+03], dtype=float32),
 array([1.], dtype=float32))

In [91]:
print('\nWith Tensor Transform')
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
type(features), type(labels), features, labels


With Tensor Transform


(torch.Tensor,
 torch.Tensor,
 tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

In [93]:
print('\nWith Tensor and Multiplication Transform')
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
type(features), type(labels), features, labels


With Tensor and Multiplication Transform


(torch.Tensor,
 torch.Tensor,
 tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
         1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
         4.2600e+03]),
 tensor([1.]))